## tensorflow 经典方法学习

### 引入 必要包

In [1]:
import tensorflow as tf
import numpy as np

d:\progrom\python\python\python3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### tf.nn.embedding_lookup函数的用法

tf.nn.embedding_lookup函数的用法主要是选取一个张量里面索引对应的元素。tf.nn.embedding_lookup（tensor, id）:tensor就是输入张量，id就是张量对应的索引，其他的参数不介绍。

参考网址： [tf.nn.embedding_lookup函数的用法](https://blog.csdn.net/uestc_c2_403/article/details/72779417)

In [11]:
c = np.random.random([10,1])
b = tf.nn.embedding_lookup(c, [1, 3])

import tensorflow as tf;
import numpy as np;
 
c = np.random.random([10,1])
b = tf.nn.embedding_lookup(c, [1, 3])
 
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    print(sess.run(b))
    print(c)


[[0.96171387]
 [0.96199781]]
[[0.77322859]
 [0.96171387]
 [0.71319904]
 [0.96199781]
 [0.46039099]
 [0.42424624]
 [0.48325266]
 [0.92541773]
 [0.10654726]
 [0.93055393]]


### 【TensorFlow】tf.concat的用法

tf.concat(concat_dim, values, name='concat')
除去name参数用以指定该操作的name，与方法有关的一共两个参数：

<b>第一个参数concat_dim</b>：必须是一个数，表明在哪一维上连接

1、如果concat_dim是0，那么在某一个shape的第一个维度上连，对应到实际，就是叠放到列上

```   
    t1 = [[1, 2, 3], [4, 5, 6]]
    t2 = [[7, 8, 9], [10, 11, 12]]
    tf.concat(0, [t1, t2]) == > [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]
```

2、 如果concat_dim是1，那么在某一个shape的第二个维度上连

```
    t1 = [[1, 2, 3], [4, 5, 6]]
    t2 = [[7, 8, 9], [10, 11, 12]]
    tf.concat(1, [t1, t2]) ==> [[1, 2, 3, 7, 8, 9], [4, 5, 6, 10, 11, 12
```

<b>第二个参数values</b>：就是两个或者一组待连接的tensor了

参考网址： [【TensorFlow】tf.concat的用法](https://blog.csdn.net/mao_xiao_feng/article/details/53366163)

### 【TensorFlow】用expand_dim()来增加维度

TensorFlow中，想要维度增加一维，可以使用tf.expand_dims(input, dim, name=None)函数。

<b>Args</b>: 
input: A Tensor. 
dim: A Tensor. Must be one of the following types: int32, int64. 0-D (scalar). Specifies the dimension index at which to expand the shape of input. 
name: A name for the operation (optional).

<b>Returns</b>: 
A Tensor. Has the same type as input. Contains the same data as input, but its shape has an additional dimension of size 1 added.

参考网址： [TensorFlow用expand_dim()来增加维度](https://blog.csdn.net/jasonzzj/article/details/60811035)

### 【TensorFlow】tf.nn.conv2d()使用

```
    conv2d(
        input,
        filter,
        strides,
        padding,
        use_cudnn_on_gpu=None,
        data_format=None,
        name=None
    )
```
1. input是一个4d输入[batch_size, in_height, in_width, n_channels]，表示图片的批数，大小和通道。
1. filter是一个4d输入[filter_height, filter_width, in_channels, out_channels]，表示kernel的大小，输入通道数和输出通道数，其中输出通道数表示从上一层提取多少特征。
1. strides是一个1d输入，长度为4，其中stride[0]和stride[3]必须为1，一般格式为[1, stride[1], stride[2], 1]，在大部分情况下，因为在height和width上的步进设为一样，因此通常为[1, stride, stride, 1]。 
1. padding是一个字符串输入，分为SAME和VALID分别表示是否需要填充，因为卷积完之后因为周围的像素没有卷积到，因此一般是会出现卷积完的输出尺寸小于输入的现象的

参考网址： 
1. [tf.nn.conv2d()使用](https://blog.csdn.net/loseinvain/article/details/78935192)
1. [TensorFlow tf.nn.conv2d()介绍](https://blog.csdn.net/guoyunfei20/article/details/78413365)

In [43]:
word_dim = 10
pos_dim = 5
hidden_dim = 6
sen_len = 10

word_embedding = np.random.random([sen_len,10,word_dim])
pos_e1_embedding = np.random.random([sen_len,10,pos_dim])
pos_e2_embedding = np.random.random([sen_len,10,pos_dim])

input_word = [1,3]
input_pos_e1 = [1,5]
input_pos_e2 = [1,2]

inputs_forward = tf.concat(axis=2, values=[tf.nn.embedding_lookup(word_embedding, input_word),\
                                           tf.nn.embedding_lookup(pos_e1_embedding, input_pos_e1), \
                                           tf.nn.embedding_lookup(pos_e2_embedding, input_pos_e2)])

inputs_forward_new = tf.Variable(tf.expand_dims(inputs_forward, -1), dtype = np.float64)  #-1表示最后一维

window=3
filter_data = tf.Variable( np.random.rand(window, word_dim + 2 * pos_dim, 1, hidden_dim), dtype = np.float64)


conv = tf.nn.conv2d(
    inputs_forward_new,
    filter_data,
    strides=[1, 1, 1, 1],
    padding='VALID',
    name='conv')

b = tf.Variable( np.random.rand(hidden_dim), dtype = np.float64)
h = tf.nn.bias_add(conv, b)

pooled = tf.nn.max_pool(
        h,
        ksize=[1, sen_len - window + 1, 1, 1],
        strides=[1, 1, 1, 1],
        padding='VALID',
        name='pool')

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    print("---------sess.run(inputs_forward)----------")
    print(sess.run(inputs_forward))
    print("---------sess.run(inputs_forward_new)----------")
    print(sess.run(inputs_forward_new))
    print("---------sess.run(w)----------")
    print(sess.run(w))
    print("---------sess.run(conv)----------")
    print(sess.run(conv))
    print("---------sess.run(b)----------")
    print(sess.run(b))
    print("---------sess.run(h)----------")
    print(sess.run(h))
    print("---------sess.run(pooled)----------")
    print(sess.run(pooled))

---------sess.run(inputs_forward)----------
[[[8.96570443e-01 8.31685654e-02 7.46792569e-01 1.06018532e-01
   7.26397135e-01 5.44089672e-01 4.82173516e-01 9.33843741e-03
   2.89801706e-01 4.10119423e-02 9.58390149e-01 4.12560511e-01
   8.83796577e-01 1.10268567e-01 9.19881510e-01 5.78728472e-01
   5.25630857e-01 5.28832557e-01 5.56670677e-02 1.57373812e-01]
  [1.04018237e-01 4.81320467e-01 5.27650978e-01 8.57054512e-01
   7.40748740e-01 1.09338858e-01 1.00325901e-01 1.89790440e-01
   7.61290041e-02 3.29477112e-01 4.92564692e-01 2.09466067e-01
   6.62136143e-01 4.56833630e-01 1.13651210e-01 7.64574761e-01
   8.75553815e-01 2.57104266e-01 3.82199989e-01 8.88400448e-01]
  [1.92234324e-01 7.31839752e-01 8.13085637e-01 7.11293615e-01
   9.63730778e-01 3.37759737e-01 3.34531565e-01 8.47761540e-01
   7.74562021e-02 9.49451964e-01 9.75115905e-01 4.89986068e-01
   5.10260496e-01 9.94694069e-01 9.91884385e-01 2.85520317e-01
   5.39394544e-01 4.71611336e-01 3.33974081e-01 7.89076839e-01]
  [9.929